In [ ]:
#!pip install crewai crewai_tools langchain==0.2.6 langchain-core==0.2.15 langchain-experimental==0.0.62 langchain-community==0.2.6 langchain_mistralai

In [1]:
from dotenv import load_dotenv
import os
local=True
load_dotenv(".env_generic")
load_dotenv(".env_neo4j_local")
model_file=".env_mistal_ai"
if (local):
    model_file=".env_ollama"
    model_file=".env_codestral" 
load_dotenv(model_file)
print(os.getenv('OPENAI_MODEL_NAME'),os.getenv('OPENAI_API_BASE'))

codestral-latest https://codestral.mistral.ai/v1/


In [2]:
from langchain_openai import ChatOpenAI
from langchain.agents import Tool
from langchain_community.graphs import Neo4jGraph
from langchain.chains.graph_qa.cypher import GraphCypherQAChain
import os
graph = Neo4jGraph(url=os.environ['NEO4J_URI'], username=os.environ['NEO4J_USERNAME'], password=os.environ['NEO4J_PASSWORD'])
graph.refresh_schema()
llm = ChatOpenAI(model=os.getenv('OPENAI_MODEL_NAME'), temperature=0)
#llm = ChatOpenAI(model=os.getenv('OPENAI_MODEL_NAME'), temperature=0)
neo4jquery = GraphCypherQAChain.from_llm(
    llm, graph=graph, verbose=True
)
CYPHER_GENERATION_PROMPT="""you must use only entities that you see in the schema. dont invent anything. you must try mode than five times few different approaches before you give up and return with no result"""
neo4j_query_tool = Tool(
  name="neo4j_query_tool",
  func=neo4jquery.run,
  description="neo4j database that contains data about the project, user provide english question the tool translates it to query and respond with the answer",
  cypher_prompt=CYPHER_GENERATION_PROMPT
)
neo4j_query_tool.run("how many floors are there?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (b:IfcBuildingStorey)
RETURN COUNT(b)
Full Context:
[{'COUNT(b)': 12}]

> Finished chain.


"I'm not able to determine the number of floors based on the provided information. The information provided is about a count of 12, but it's not specified what that count refers to. Without more context or information, I can't accurately answer the question about the number of floors."

In [5]:
# Agent 1: Researcher
from crewai import Agent, Task, Crew
analyst = Agent(
    role="civil engineer",
    goal="get information about materials from data tool you ask the tool the questions in human language",
    tools = [  neo4j_query_tool],
    verbose=True,
    backstory=(
        "As civil engineer, you get your input from the neo4j_query_tool, you ask the questions in human language, for example, if you want to know number of floors you ask : how many floors are there? "
        "you might need to calculate things by yourself, according to the data you get from the tool. you are providing quantities of materials needed. for example: number of bricks of each kind "
        "you must be very accurate and dont miss anything. dont invent anything. you must get approval from senior civil engineer, before submitting the answer. you can also consult with him for any issue "
        "if the supervisor is wrong, tell him he is wrong"
    )
)
senior = Agent(
    role="senior civil engineer",
    goal="validate that the civil engineer is doing proper work. ",
    tools = [  neo4j_query_tool],
    verbose=True,
    backstory=(
        "As senior civil engineer, you are validating that the data provided by the engineer is correct. beaware that he has the tendency to miss thing and invent things. "
        "This is not acceptable, and you must follow each step he does and make sure it was correct. he can consult with you when needed, you are guru in IFC and very experienced"
    )
)



# Task for Researcher Agent:
analyst_task = Task(
    description=(
        "the data is about building, need to get list of walls and all the information you have on the material they are made of " 
        "sometimes the type of the bricks can be concluded from the Name property of the wall. for example, wall named 'Basic Wall:Concrete 24', means that its made of Concrete 24 brick "
        " if you dont find the full data provide list of walls you did find the data "
        "you must try mode than five times few different approaches before you give up"
    ),
    expected_output=(
        "list of walls, each wall separately with duplications,  their type (for example concrete 11 )"
    ),
    agent=analyst,
    async_execution=False
)

senior_task = Task(
    description=(
        "supervise any step that the civil engineer is doing, and cross validate he does descent job. i "
        "if he does mistake, correct him and ask him to do again with your hints, the chain should not be stopped till final correct answer is submitted "
        "sometimes the type of the bricks can be concluded from the Name property of the wall. for example, wall named 'Basic Wall:Concrete 24', means that its made of Concrete 24 brick "
    ),
    expected_output=(
        "The findings provided by the civil engineer with your approval, in json format, having the wall id and its type"
    ),
    agent=senior,
    async_execution=False
)




application_crew = Crew(
    agents=[analyst, senior],

    tasks=[analyst_task,senior_task ],

    verbose=True
)
result = application_crew.kickoff()


2024-07-23 19:49:53,435 - 140704580679424 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-07-23 19:49:53][DEBUG]: == Working Agent: civil engineer
 [2024-07-23 19:49:53][INFO]: == Starting Task: the data is about building, need to get list of walls and all the information you have on the material they are made of sometimes the type of the bricks can be concluded from the Name property of the wall. for example, wall named 'Basic Wall:Concrete 24', means that its made of Concrete 24 brick  if you dont find the full data provide list of walls you did find the data you must try mode than five times few different approaches before you give up


> Entering new CrewAgentExecutor chain...
I will first ask the neo4j_query_tool to provide me with a list of all the walls in the database. Then, I will try to extract the type of material from the properties of each wall, specifically focusing on the 'Name' property as it often contains information about the brick type.

Action: neo4j_query_tool
Action Input: {"tool_input": "MATCH (w:Wall) RETURN w.Name AS WallName, w.Type AS Wall

In [ ]:
#from langchain_mistralai.chat_models import ChatMistralAI
#llm = ChatMistralAI(api_key="YeBPYL5uYm1mZtYI8d0iAoN7Hwajwd2I", base_url="https://codestral.mistral.ai/v1/fim/completions",model="codestral-latest",temperature=0)
#import requests
# Define the non-default URL
#api_url = 'https://codestral.mistral.ai/v1/chat/completions'
# Setup the request headers and data
#headers = {
#    'Authorization': 'Bearer ',
#    'Content-Type': 'application/json'
#}
#data = {
#    'model': 'codestral-latest',
#    'messages': [
#        {'role': 'user', 'content': 'What is the best French cheese?'}
#    ]
#}

# Make the POST request to the non-default URL
#response = requests.post(api_url, headers=headers, json=data)

# Print the response
#print(response.json())

In [ ]:
#from langchain_mistralai.chat_models import ChatMistralAI
#from langchain_core.messages import HumanMessage

# Define the correct endpoint for the pre-GA model
#endpoint = 'https://codestral.mistral.ai/v1/'

# Initialize the ChatMistralAI client with the custom endpoint
#chat = ChatMistralAI(api_key=os.environ['OPENAI_API_KEY'], model=os.getenv('OPENAI_MODEL_NAME'),endpoint = os.getenv('OPENAI_API_BASE') )
# Define the messages to send
#messages = [HumanMessage(content="knock knock")]

# Invoke the chat completion
#response = chat.invoke(messages)

# Print the response
#print(response)